# CS 4650 Final Project

#### I am planning on doing more research to develop and test my stocks algorithm. I still need to do more studying; however, I did develop some code for getting live data from the market, which is below.

In [5]:
pip install yfinance

  Using cached lxml-4.6.4-cp310-cp310-win_amd64.whl (3.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=f0c0fb99a260d9842d4a7faeed73bf227a3ccc163b8ff72d445b7df145705052
  Stored in directory: c:\users\juanj\appdata\local\pip\cache\wheels\94\f0\94\cb5d98a5536d62811bb509b22bd2597d5bd7154752299383d3
Successfully built multitasking
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go

In [11]:
#The Yahoo API needs the following inputs: Tickers, Start and End Date (or Period), and Interval
# period is 1 week for this example and interval is one minute from which we will get our data
AppleData = yf.download(tickers = 'AAPL', period = '1wk', interval = '1m')

#Then we print the data
AppleData

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-15 09:30:00-05:00,150.369995,150.800003,150.289993,150.779999,150.779999,2052956
2021-11-15 09:31:00-05:00,150.809998,151.220001,150.789993,151.179993,151.179993,615526
2021-11-15 09:32:00-05:00,151.190002,151.429993,151.119995,151.156204,151.156204,521593
2021-11-15 09:33:00-05:00,151.164993,151.360001,151.139999,151.270004,151.270004,334812
2021-11-15 09:34:00-05:00,151.250000,151.374207,151.229996,151.320007,151.320007,363213
...,...,...,...,...,...,...
2021-11-19 15:55:00-05:00,160.324997,160.380005,160.309998,160.339996,160.339996,524498
2021-11-19 15:56:00-05:00,160.339996,160.399994,160.320007,160.320007,160.320007,478400
2021-11-19 15:57:00-05:00,160.315002,160.490005,160.315002,160.479996,160.479996,688592
